## General Workflow

### 1) Identifying Business Problem
### 2) Data Preprocessing
### 3) Exploratory Data Analysis (EDA)

### 1) Identifying Business Problem

In this case, we're asked to develop a model for an organization that helps non-profit organizations to find donors. This model will be used to predict whether a donor will spend more than or equal to 30€, given that this donor receives a re-activation campaign. The model will be trained based on the previous campaign, and then validated with another data from other campaign.

The model is expected to have a good performance and high interpretability. In addition, we also have to emphasize why does implementing the model is better than random guessing campaign. In the end, the model will be used to predict potential customers for the upcoming campaign, and which customers are the most effective to reach.

### 2) Data Preprocessing

Our data consist of 5 csv files which includes the donor database, the amount of donated, and the selected donors during previous campaign

In [59]:
import pandas as pd
import numpy as np
import datetime

In [60]:
# Drop 1st Column
donors=pd.read_csv("./data/donors.csv")

donors['dateOfBirth'] = pd.to_datetime(donors['dateOfBirth'])

donors['province'] = donors['province'].apply(lambda row: 'Antwerp' if row == 'ANTWERP' else row)

gender_dict={'F':1,'M':0,'NaN':969}
language_dict={'EN':1,'FR':2,'NL':3}
region_dict={'Brussels':1,'Flanders':2,'Missing':3,'Wallonia':4}
province_dict={'Antwerp':1,'Brussels':2,'East Flanders':3,'Flemish Brabant':4,
                'Hainaut':5,'Liege':6,'Limburg':7,'Luxembourg':8,'Missing':9,
                'Namur':10,'Walloon Brabant':11,'West Flanders':12}


donors = donors.replace({"province": province_dict, "region": region_dict, "gender": gender_dict,"language":language_dict})

donors.head()

,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,4,2,1.0,3,1967-12-24
1,1,100002,2260.0,1,2,0.0,3,1952-01-14
2,2,100003,1780.0,4,2,0.0,3,1986-12-23
3,3,100004,1020.0,2,1,1.0,2,1952-03-29
4,4,100005,1020.0,2,1,1.0,2,1986-06-25


In [61]:
campaigns=pd.read_csv("./Data/campaigns.csv",delimiter=";")

for i in range(0,len(campaigns)):
    campaigns.iloc[i,2]=campaigns.iloc[i,2].replace('\u202f','')

for i in range(0,len(campaigns)):
    campaigns.iloc[i,3]=campaigns.iloc[i,3].replace('\xa0€','')

for i in range(0,len(campaigns)):
    campaigns.iloc[i,3]=campaigns.iloc[i,3].replace(',','.')

campaigns['date']=pd.to_datetime(campaigns['date'])
campaigns['lettersSent']=campaigns['lettersSent'].astype(int)
campaigns['lettersSent']=campaigns['lettersSent'].astype(float)
campaigns.head()

,campaignID,date,lettersSent,CostUnit
0,153,2004-12-13,6873.0,0.17
1,154,2005-01-01,7656.0,0.30
2,180,2005-01-18,9933.0,0.20
3,433,2005-01-29,7448.0,0.59
4,476,2005-12-02,6605.0,0.51


In [62]:
gifts=pd.read_csv("./Data/gifts.csv")
#cleaning gift
# Change date into datetime object
gifts['date'] = pd.to_datetime(gifts['date'])
#
gifts['amount'] = gifts['amount'].str.replace(',', '.')
#
gifts['amount'] = gifts['amount'].astype('float')
#
gifts.head()

,Unnamed: 0,donorID,campaignID,amount,date
0,0,100001,NaN,12.39,2004-03-23
1,1,100001,1577.0,12.39,2007-10-11
2,2,100001,1634.0,6.69,2007-12-28
3,3,100001,1778.0,24.79,2008-06-14
4,4,100001,1808.0,7.44,2008-07-09


In [63]:
#
def agg_A(tr):
    tr['total_donated_during_dv'] = tr['amount'].sum()
    return tr
#
gift_6169=gifts[gifts['campaignID'] == 6169]
#
gift_6169 = gift_6169.groupby(by='donorID').apply(agg_A)
#
gift_6169=gift_6169.drop(['campaignID','Unnamed: 0','amount','date'],axis=1)
#
gift_6169.head()

,donorID,total_donated_during_dv
132,100023,15.0
705,100126,123.0
1151,100253,120.0
1362,100279,75.0
1804,100320,135.0


In [64]:
#
gift_7244=gifts[gifts['campaignID'] == 7244]
#
gift_7244 = gift_7244.groupby(by='donorID').apply(agg_A)
#
gift_7244=gift_7244.drop(['campaignID','Unnamed: 0','amount','date'],axis=1)
#
gift_7244.head()

,donorID,total_donated_during_dv
5828,100724,40.0
7527,100896,10.0
8314,100964,100.0
9269,101066,60.0
9316,101069,80.0


In [65]:
selection_6169=pd.read_csv("./Data/selection campaign 6169.csv")
selection_7244=pd.read_csv("./Data/selection campaign 7244.csv")
selection_7362=pd.read_csv("./Data/new_campaign_selection.csv")

In [66]:
#duration of data gadering
iv_period = 180
# dayes befor starting the data gadering
in_between_period = 30
# start and end dates of campaign 6169
date_campaign_6169 = datetime.datetime(2018, 9, 4)
end_period_6169 = date_campaign_6169 - datetime.timedelta(days = in_between_period)
start_period_6169 = end_period_6169 - datetime.timedelta(days = iv_period)
# start and end dates of campaign 7244
date_campaign_7244 = datetime.datetime(2019, 6, 18)
end_period_7244 = date_campaign_7244 - datetime.timedelta(days = in_between_period)
start_period_7244 = end_period_7244 - datetime.timedelta(days = iv_period)
# start and end dates of campaign 7362
date_campaign_7362 = datetime.datetime(2020, 2, 1)
end_period_7362 = date_campaign_7362 - datetime.timedelta(days = in_between_period)
start_period_7362 = end_period_7362 - datetime.timedelta(days = iv_period)


## 6169 Base Table

In [68]:
#
def agg_B(tr):
    tr['last_date_dono'] = tr['date'].max()
    tr['frequency_donor'] = tr['amount'].count()
    tr['total_donated'] = tr['amount'].sum()
    tr = tr.drop(['date','amount'],axis=1)
    return tr
#
gift_base_6169 = gifts[(gifts['date'] >= start_period_6169) &  (gifts['date'] <= end_period_6169)]

gift_base_6169=gift_base_6169.drop(['Unnamed: 0','campaignID'],axis=1)

#
gift_base_6169 = gift_base_6169.groupby(by='donorID').apply(agg_B)

# remove duplicate rows
gift_base_6169 = gift_base_6169.drop_duplicates(subset=['donorID'])


#
gift_base_6169['Recency_dono'] = end_period_6169 - gift_base_6169['last_date_dono']
#
gift_base_6169['Recency_dono'] = gift_base_6169['Recency_dono'].dt.days
#
gift_base_6169=gift_base_6169.drop(['last_date_dono'],axis=1)



# Build Base Table Based On the 6169 Campaign Data
gift_base_6169 = pd.merge(selection_6169,gift_base_6169, how='left', on='donorID')


# # Merge with dv table
gift_base_6169 = pd.merge(gift_base_6169, gift_6169, how='left', on='donorID')
# Merge with donor database
gift_base_6169 = pd.merge(gift_base_6169, donors, how='left', on='donorID')
# Target Variable
gift_base_6169['donated_more_than30_duringdv'] = gift_base_6169['total_donated_during_dv'].apply(
                                                                lambda row: 1 if row >= 30 else 0)
#
gift_base_6169['Age'] = gift_base_6169['dateOfBirth'].apply(lambda row:
                 date_campaign_6169.year - row.year -((date_campaign_6169.month, date_campaign_6169.day) < (row.month, row.day) ))
#
gift_base_6169=gift_base_6169.drop(['Unnamed: 0','total_donated_during_dv','dateOfBirth'],axis=1)
#filling nans
gift_base_6169['total_donated'] = gift_base_6169['total_donated'].fillna(gift_base_6169['total_donated'].mean())
gift_base_6169['Recency_dono'] = gift_base_6169['Recency_dono'].fillna(gift_base_6169['Recency_dono'].mean())
gift_base_6169['frequency_donor'] = gift_base_6169['frequency_donor'].fillna(gift_base_6169['frequency_donor'].median())
#save base tabel 6169 
gift_base_6169.to_csv('./data/gift_base_6169.csv')
#showing Data
gift_base_6169.head()

,donorID,frequency_donor,total_donated,Recency_dono,zipcode,province,region,gender,language,donated_more_than30_duringdv,Age
0,100001,1.0,31.778418,51.219388,1861.0,4.0,2.0,1.0,3.0,0,50.0
1,100002,1.0,31.778418,51.219388,2260.0,1.0,2.0,0.0,3.0,0,66.0
2,100004,1.0,31.778418,51.219388,1020.0,2.0,1.0,1.0,2.0,0,66.0
3,100008,1.0,31.778418,51.219388,1780.0,4.0,2.0,0.0,2.0,0,81.0
4,100009,1.0,31.778418,51.219388,1020.0,2.0,1.0,0.0,2.0,0,86.0


In [69]:
#
gift_base_7244 = gifts[(gifts['date'] >= start_period_7244) &  (gifts['date'] <= end_period_7244)]

#
gift_base_7244=gift_base_7244.drop(['campaignID','Unnamed: 0'],axis=1)

#
gift_base_7244=gift_base_7244.groupby(by='donorID').apply(agg_B)

# remove duplicate rows
gift_base_7244 = gift_base_7244.drop_duplicates(subset=['donorID'])

#
gift_base_7244['Recency_dono'] = end_period_7244 - gift_base_7244['last_date_dono']
#
gift_base_7244['Recency_dono'] = gift_base_7244['Recency_dono'].dt.days
#
gift_base_7244=gift_base_7244.drop(['last_date_dono'],axis=1)
# Build Base Table Based On the 7244 Campaign Data
gift_base_7244 = pd.merge(selection_7244,gift_base_7244, how='left', on='donorID')
# # Merge with iv table
gift_base_7244 = pd.merge(gift_base_7244, gift_7244, how='left', on='donorID')
# Merge with donor database
gift_base_7244 = pd.merge(gift_base_7244, donors, how='left', on='donorID')
# Target Variable
gift_base_7244['donated_more_than30_duringdv'] = gift_base_7244['total_donated_during_dv'].apply(
                                                                lambda row: 1 if row >= 30 else 0)
#
gift_base_7244['Age'] = gift_base_7244['dateOfBirth'].apply(lambda row:
                 date_campaign_7244.year - row.year -((date_campaign_7244.month, date_campaign_7244.day) < (row.month, row.day) ))
#
gift_base_7244=gift_base_7244.drop(['Unnamed: 0','total_donated_during_dv','dateOfBirth'],axis=1)
#filling nans
gift_base_7244['total_donated'] = gift_base_7244['total_donated'].fillna(gift_base_7244['total_donated'].mean())
gift_base_7244['Recency_dono'] = gift_base_7244['Recency_dono'].fillna(gift_base_7244['Recency_dono'].mean())
gift_base_7244['frequency_donor'] = gift_base_7244['frequency_donor'].fillna(gift_base_7244['frequency_donor'].median())
#save base tabel 7244
gift_base_7244.to_csv('./data/gift_base_7244.csv')
#showing Data
gift_base_7244.head()

,donorID,frequency_donor,total_donated,Recency_dono,zipcode,province,region,gender,language,donated_more_than30_duringdv,Age
0,100001,1.0,42.550312,53.640625,1861.0,4,2,1.0,3,0,51
1,100002,1.0,42.550312,53.640625,2260.0,1,2,0.0,3,0,67
2,100008,1.0,42.550312,53.640625,1780.0,4,2,0.0,2,0,81
3,100009,1.0,42.550312,53.640625,1020.0,2,1,0.0,2,0,87
4,100010,1.0,42.550312,53.640625,1020.0,2,1,0.0,2,0,36


In [70]:
#
gift_base_7362 = gifts[(gifts['date'] >= start_period_7362) &  (gifts['date'] <= end_period_7362)]

#
gift_base_7362=gift_base_7362.drop(['campaignID','Unnamed: 0'],axis=1)

#
gift_base_7362=gift_base_7362.groupby(by='donorID').apply(agg_B)

# remove duplicate rows
gift_base_7362 = gift_base_7362.drop_duplicates(subset=['donorID'])

#
gift_base_7362['Recency_dono'] = end_period_7362 - gift_base_7362['last_date_dono']
#
gift_base_7362['Recency_dono'] = gift_base_7362['Recency_dono'].dt.days
#
gift_base_7362=gift_base_7362.drop(['last_date_dono'],axis=1)
# Build Base Table Based On the 7362 Campaign Data
gift_base_7362 = pd.merge(selection_7362,gift_base_7362, how='left', on='donorID')
# Merge with donor database
gift_base_7362 = pd.merge(gift_base_7362, donors, how='left', on='donorID')

gift_base_7362['Age'] = gift_base_7362['dateOfBirth'].apply(lambda row:
                 date_campaign_7362.year - row.year -((date_campaign_7362.month, date_campaign_7362.day) < (row.month, row.day) ))
#
gift_base_7362=gift_base_7362.drop(['Unnamed: 0','dateOfBirth'],axis=1)
#filling nans
gift_base_7362['total_donated'] = gift_base_7362['total_donated'].fillna(gift_base_7362['total_donated'].mean())
gift_base_7362['Recency_dono'] = gift_base_7362['Recency_dono'].fillna(gift_base_7362['Recency_dono'].mean())
gift_base_7362['frequency_donor'] = gift_base_7362['frequency_donor'].fillna(gift_base_7362['frequency_donor'].median())
#save base tabel 7362
gift_base_7362.to_csv('./data/gift_base_7362.csv')
#showing Data
gift_base_7362.head()

,donorID,frequency_donor,total_donated,Recency_dono,zipcode,province,region,gender,language,Age
0,134242,1.0,43.575577,71.601703,3580.0,7,2,0.0,3,53
1,126503,1.0,43.575577,71.601703,4250.0,6,4,0.0,2,70
2,104392,1.0,43.575577,71.601703,4820.0,6,4,1.0,2,45
3,105905,1.0,55.000000,34.000000,8370.0,12,2,0.0,3,49
4,120506,1.0,43.575577,71.601703,8020.0,12,2,0.0,3,46
